In [ ]:
!pip install transformers datasets torch

In [6]:
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
import time

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
dataset = load_dataset('wmt/wmt19', 'de-en', split='validation')

In [8]:
print(dataset['translation'][0]['en'])

Munich 1856: Four maps that will change your view of the city


In [10]:
model_name = 't5-large'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
model = model.to(device)

In [24]:
def translate_batch(input_ids):
    with torch.no_grad():
        outputs = model.generate(input_ids)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

inputs = tokenizer(["translate English to German: " + entry['en'] for entry in dataset['translation']],
                   return_tensors="pt", padding=True, truncation=True)

inputs = inputs.to(device)

In [25]:
batch_size = 16  # Adjust based on memory availability
total_inference_time = 0

start_time = time.time()

translations = []

for i in range(0, len(dataset['translation']), batch_size):
    batch_input_ids = inputs.input_ids[i:i + batch_size]

    translations.extend(translate_batch(batch_input_ids))

end_time = time.time()

total_inference_time = end_time - start_time

print(f'Total inference time : {total_inference_time}s')

Total inference time : 199.79150891304016s


In [23]:
for i, (original, translated) in enumerate(zip([entry['en'] for entry in dataset['translation'][:10]], translations[:10])):
    print(f"Original: {original} => Translated: {translated}")

Original: Munich 1856: Four maps that will change your view of the city => Translated: München 1856: Vier Karten, die Ihre Sicht auf die Stadt verändern
Original: A mental asylum, where today young people are said to meet. => Translated: Ein psychisches Asyl, wo sich heute junge Menschen treffen sollen.
Original: A crypt chapel, where they are now digging tunnels for the S-Bahn. => Translated: Eine Krypta, wo jetzt Tunnel für die S-Bahn gegraben
Original: Allotment holders cultivate the soil of former farmers. => Translated: Die Besitzer von Feldern bewirtschaften den Boden ehemaliger Bauern.
Original: The oldest official map of Munich brings captivating stories to light. => Translated: Die älteste offizielle Karte von München erfährt spannende Geschichten.
Original: It is annoying when geographical maps are not up-to-date. => Translated: Es ist ärgerlich, wenn die geographischen Karten nicht auf dem neuesten Stand
Original: Anyone who has ever got worked up because the car's sat-nav i